<a href="https://colab.research.google.com/github/ggnanaraj/Story/blob/main/Stock0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nselib

In [2]:
pip install numpy

In [ ]:
import numpy as np

In [ ]:
import nselib
#data = nselib.trading_holiday_calendar()
#print(data)

In [ ]:
from nselib import capital_market

In [ ]:
data=capital_market.bhav_copy_with_delivery("01-03-2024")
print(data.columns)

Index(['SYMBOL', 'SERIES', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE',
       'CLOSE_PRICE', 'PREV_CLOSE', 'TTL_TRD_QNTY', 'TURNOVER_LACS',
       'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER', 'DATE1'],
      dtype='object')


In [ ]:
#data1=data('SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER')
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)


In [5]:
import nselib
from nselib import capital_market
data=capital_market.bhav_copy_with_delivery("07-03-2024")
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)
data1['TTL_TRD_QNTY']=0
data1['DELIV_PER']=0

#data1=data1[data1['DELIV_PER'] != "-"]
data_5PC_GAP_UP=data1[data1['GAP_UP'] > 5]     # gap greater than 5
data_5PC_GAP_UP=data_5PC_GAP_UP.sort_values(by=['GAP_UP'],ascending=False)
data_PC=data_5PC_GAP_UP[data_5PC_GAP_UP['PREV_CLOSE'] > 10] # only share value above Rs10
data_PC.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate
data_PC=data_PC.sort_values(by=['SYMBOL'])
list_SYM=data_PC['SYMBOL'].to_list()
list_SYM.sort()
#---------------------------------
dataO=capital_market.bhav_copy_with_delivery("06-03-2024")
data1O=dataO.loc[1:,['SYMBOL','TTL_TRD_QNTY','DELIV_PER']]
data1O=data1O[data1O['DELIV_PER'] != "-"]
data2O=data1O[data1O['SYMBOL'].isin(list_SYM)]# list_SYM or data_PC or list_PC
data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate
data2O=data2O.sort_values(by=['SYMBOL'])
listO_SYM=data2O['SYMBOL'].to_list()
listO_SYM.sort()
#---------------------------
list_comn=[symbol for symbol in list_SYM if symbol in listO_SYM]
list_diff=[symbol for symbol in list_SYM if symbol not in listO_SYM]
list_indexDel=[]
data_PC1= data_PC
for i in list_diff:
  list_1=data_PC.index[data_PC['SYMBOL'] == i ].to_list()
  data_PC1.drop(list_1, inplace=True) # drops row which is not common
  list_indexDel.extend(list_1)
for j,k in enumerate(list_comn):
  PC1=data2O.loc[data2O['SYMBOL']==list_comn[j],'TTL_TRD_QNTY']
  data_PC1.loc[data_PC1['SYMBOL']==list_comn[j],'TTL_TRD_QNTY']= int(PC1)
  PC1=data2O.loc[data2O['SYMBOL']==list_comn[j],'DELIV_PER']
  data_PC1.loc[data_PC1['SYMBOL']==list_comn[j],'DELIV_PER']= float(PC1)

#data2 = capital_market.price_volume_and_deliverable_position_data(symbol='SBIN', from_date='29-02-2024', to_date='01-03-2024')
#data3=capital_market.equity_list( )
data_VOL=data_PC1[data_PC1['TTL_TRD_QNTY'] > 50000] #Quantity greater than 50000
data_VOL=data_VOL[data_VOL['DELIV_PER'] != "-"]
#data_VOL=data_VOL[data_VOL['DELIV_PER'] > 20]  #
#data_VOL=data_VOL[data_VOL['DELIV_PER'] < 90]
list_N=data_VOL['SYMBOL'].to_list()
data_VOL['GAP_down']=(((data_VOL['HIGH_PRICE']-data_VOL['LOW_PRICE'])/data_VOL['HIGH_PRICE'])*100).round(2)
data_VOL['INT_Hike']=(((data_VOL['HIGH_PRICE']-data_VOL['OPEN_PRICE'])/data_VOL['OPEN_PRICE'])*100).round(2)
#data_VOL


<ipython-input-5-96838e231a93>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_PC.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate
<ipython-input-5-96838e231a93>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate


In [16]:
#print(len(data_PC1))
#print(len(data2O))
data_VOL1=[]
Hi_Max1=[]
Hi_Min1=[]
Hi_Ave1=[]
Lo_Max1=[]
Lo_Min1=[]
Lo_Ave1=[]
for i in range(len(list_N)):
  data_T=capital_market.price_volume_data(list_N[i],period='1M').tail(14)
  #print(data_T)
  #-------------------------------------------------
  data_Tr=data_T[:-1]
  data_Tr= data_Tr.tail(7)
  Hi_Max=data_Tr['HighPrice'].max()
  Hi_Min=data_Tr['HighPrice'].min()

  data_Hi=data_Tr['HighPrice'].astype('string')
  for j in range(len(data_Hi)):
    data_Hi.iloc[j]=data_Hi.iloc[j].replace(",","")
  data_Hi=data_Hi.astype('string').astype(float)
  #print(data_T)
  Hi_Ave=data_Hi.mean(skipna=True).astype(int)


  Lo_Max=data_Tr['LowPrice'].max()
  Lo_Min=data_Tr['LowPrice'].min()

  data_Lo=data_Tr['LowPrice'].astype('string')
  for j in range(len(data_Lo)):
    data_Lo.iloc[j]=data_Lo.iloc[j].replace(",","")
  data_Lo=data_Lo.astype('string').astype(float)
  #print(data_T)
  Lo_Ave=data_Lo.mean(skipna=True).astype(int)

  #-------------------------------------------------
  data_T=data_T['TotalTradedQuantity'].astype('string')
  for j in range(len(data_T)):
    data_T.iloc[j]=data_T.iloc[j].replace(",","")
  data_T=data_T.astype(int)
  data_T = data_T[:-1]
  #print(data_T)
  data_T1=data_T.mean(skipna=True).astype(int)
  data_VOL1.append(data_T1)
  Hi_Max1.append(Hi_Max)
  Hi_Ave1.append(Hi_Ave)
  Hi_Min1.append(Hi_Min)
  Lo_Max1.append(Lo_Max)
  Lo_Ave1.append(Lo_Ave)
  Lo_Min1.append(Lo_Min)
data_VOL['Ave_QNTY']= data_VOL1
data_VOL['Hi_Max']= Hi_Max1
data_VOL['Hi_Ave']= Hi_Ave1
data_VOL['Hi_Min']= Hi_Min1
data_VOL['Lo_Max']= Lo_Max1
data_VOL['Lo_Ave']= Lo_Ave1
data_VOL['Lo_Min']= Lo_Min1
#data_VOL
#list_comn[1]

"""data_T=capital_market.price_volume_data(list_comn[1],period='1M').tail(14)
data_T=data_T['TotalTradedQuantity'].astype('string')
for i in range(len(data_T)):
  data_T.iloc[i]=data_T.iloc[i].replace(",","")
data_T=data_T.astype(int)
data_T.mean(skipna=True).astype(int)"""

'data_T=capital_market.price_volume_data(list_comn[1],period=\'1M\').tail(14)\ndata_T=data_T[\'TotalTradedQuantity\'].astype(\'string\')\nfor i in range(len(data_T)):\n  data_T.iloc[i]=data_T.iloc[i].replace(",","")\ndata_T=data_T.astype(int)\ndata_T.mean(skipna=True).astype(int)'

In [ ]:
#print(len(data_PC1))
#print(len(data2O))
data_VOL1=[]
Hi_Max1=[]
Hi_Min1=[]
Hi_Ave1=[]
Lo_Max1=[]
Lo_Min1=[]
Lo_Ave1=[]
for i in range(len(list_N)):
  data_T=capital_market.price_volume_data(list_N[i],period='1M').tail(14)
  #print(data_T)
  #-------------------------------------------------
  data_Tr=data_T[:-1]
  data_Tr= data_Tr.tail(7)
  Hi_Max=data_Tr['HighPrice'].max()
  Hi_Min=data_Tr['HighPrice'].min()

  data_Hi=data_Tr['HighPrice'].astype('string')
  for j in range(len(data_Hi)):
    data_Hi.iloc[j]=data_Hi.iloc[j].replace(",","")
  data_Hi=data_Hi.astype('string').astype(float)
  #print(data_T)
  Hi_Ave=data_Hi.mean(skipna=True).astype(int)


  Lo_Max=data_Tr['LowPrice'].max()
  Lo_Min=data_Tr['LowPrice'].min()

  data_Lo=data_Tr['LowPrice'].astype('string')
  for j in range(len(data_Lo)):
    data_Lo.iloc[j]=data_Lo.iloc[j].replace(",","")
  data_Lo=data_Lo.astype('string').astype(float)
  #print(data_T)
  Lo_Ave=data_Lo.mean(skipna=True).astype(int)

  #-------------------------------------------------
  data_T=data_T['TotalTradedQuantity'].astype('string')
  for j in range(len(data_T)):
    data_T.iloc[j]=data_T.iloc[j].replace(",","")
  data_T=data_T.astype(int)
  data_T = data_T[:-1]
  #print(data_T)
  data_T1=data_T.mean(skipna=True).astype(int)
  data_VOL1.append(data_T1)
  Hi_Max1.append(Hi_Max)
  Hi_Ave1.append(Hi_Ave)
  Hi_Min1.append(Hi_Min)
  Lo_Max1.append(Lo_Max)
  Lo_Ave1.append(Lo_Ave)
  Lo_Min1.append(Lo_Min)
data_VOL['Ave_QNTY']= data_VOL1
data_VOL['Hi_Max']= Hi_Max1
data_VOL['Hi_Ave']= Hi_Ave1
data_VOL['Hi_Min']= Hi_Min1
data_VOL['Lo_Max']= Lo_Max1
data_VOL['Lo_Ave']= Lo_Ave1
data_VOL['Lo_Min']= Lo_Min1
data_VOL
#list_comn[1]


In [17]:
"""PC1=data2O.loc[data2O['SYMBOL']==listO_SYM[0],'TTL_TRD_QNTY']
data_PC1.loc[data_PC1['SYMBOL']==listO_SYM[0],'TTL_TRD_QNTY']= int(PC1)
PC1=data2O.loc[data2O['SYMBOL']==listO_SYM[0],'DELIV_PER']
data_PC1.loc[data_PC1['SYMBOL']==listO_SYM[0],'DELIV_PER']= float(PC1)
#data2O.loc[data2O['SYMBOL']==listO_SYM[0],'TTL_TRD_QNTY']
#data_PC1['TTL_TRD_QNTY'].astype('Int64')
#PC1"""
#data_Tr['HighPrice']=data_Tr['HighPrice'].astype('string')
#data_Tr['HighPrice']=data_Tr['HighPrice'].astype(float)
#data_Tr.dtypes
#list_N
L=[]
#data_Tr['HighPrice'].mean()
L=data_Tr['HighPrice'].to_list()
#print(L)
#data_Tr['HighPrice'].iloc[0]+ data_Tr['HighPrice'].iloc[1]
#np.array(L)
data_VOL



,SYMBOL,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRICE,PREV_CLOSE,GAP_UP,TTL_TRD_QNTY,DELIV_PER,GAP_down,INT_Hike,Ave_QNTY,Hi_Max,Hi_Min,Lo_Max,Lo_Min,Hi_Ave,Lo_Ave
522,DATAPATTNS,2797.70,2797.70,2665.00,2698.10,2634.40,6.20,410898,32.49,4.74,0.00,697400,"2,939.85","2,401.55","2,665.00","2,101.00",2668,2441
817,GPTINFRA,188.00,188.00,176.50,178.45,172.75,8.83,324877,48.03,6.12,0.00,186824,192.1,156.95,183.05,137.0,178,158
1208,JWL,398.00,398.00,368.00,369.90,362.05,9.93,979845,57.34,7.54,0.00,1469830,398.0,339.35,368.0,300.5,370,342
1340,LEXUS,43.80,44.90,42.40,44.90,40.85,7.22,237734,92.59,5.57,2.51,59787,47.8,40.0,42.5,35.1,43,38
1536,MOS,168.80,168.80,157.00,160.00,159.00,6.16,262400,58.54,6.99,0.00,225969,170.5,139.5,157.0,123.1,156,141
1823,PRECISION,52.85,52.85,51.10,51.60,50.15,5.38,60000,83.33,3.31,0.00,60461,54.0,47.8,53.1,41.5,51,47
2329,SYRMA,569.95,569.95,522.50,525.55,519.80,9.65,426928,55.66,8.33,0.00,774396,569.95,492.0,523.1,450.0,525,493
2425,TRANSTEEL,74.00,74.00,68.00,72.65,70.15,5.49,216000,83.33,8.11,0.00,134307,74.55,65.0,72.95,59.0,71,65
2521,VENUSREM,345.80,345.80,330.55,334.20,327.60,5.56,121999,42.23,4.41,0.00,82520,350.5,319.0,336.0,281.1,338,315
2596,WSI,146.95,167.85,141.05,150.50,139.90,5.04,349866,64.34,15.97,14.22,271768,167.85,134.7,142.85,124.8,149,134
